In [3]:
from abc import ABC

import numpy as np

from typing import List, Tuple

In [4]:
class Trackable(ABC):
    TRACKABLE_FIELDS = ()
    
    def __init__(self, 
                 *args,
                 display_name: str = None,
                 trackable_fields: Tuple = None):
        if display_name is None:
            raise ValueError(f'A display_name should be specified when creating a Trackable object')
        self.display_name = display_name
        
        assert isinstance(self.TRACKABLE_FIELDS, tuple)
        assert len(self.TRACKABLE_FIELDS) > 0
        
    def get_current_state(self):
        current_state = {}
        for field_name in self.TRACKABLE_FIELDS:
            current_state[field_name] = getattr(self, field_name)
            
        return current_state
        

class HistoriesStorage:
    def __init__(self):
        self.objects = {}
        self.histories = {}
        
    def register_object(self, obj: Trackable):
        assert issubclass(type(obj), Trackable)
        if (obj_name := obj.display_name) in self.objects:
            raise RuntimeError(f'An object with the name {obj_name} has been already registered')
        else:
            self.objects[obj_name] = obj
            for field_name in obj.TRACKABLE_FIELDS:
                if not hasattr(obj, field_name):
                    raise ValueError(f'An error while trying to registed an object {obj_name} with Histories. '
                                      f'The object doesn\'t have {field_name} attribute.')
                self.histories[f'{obj_name}_{field_name}'] = []
        
    def update_histories(self):
        for obj_name, obj in self.objects.items():
            current_state = obj.get_current_state()
            for field_name, field_value in current_state.items():
                self.histories[f'{obj_name}_{field_name}'].append(field_value)
                
    def flush_histories(self):
        for tracked_field_name in self.histories.keys():
            self.histories[tracked_field_name] = []

In [5]:
class IonSpecies(Trackable):
    TRACKABLE_FIELDS = ('internal_conc', )
    
    def __init__(self,
                 *,
                 internal_conc: float = 0.5,
                 external_conc: float = 0.2,
                 **kwargs):
        super().__init__(**kwargs)
        self.internal_conc = internal_conc
        self.external_conc = external_conc
        
class Vesicle(Trackable):
    TRACKABLE_FIELDS = ('area', 'volume')
    
    def __init__(self,
                 *,
                 area: float = 0.1,
                 volume: float = 0.2,
                 **kwargs):
        super().__init__(**kwargs)
        self.volume = volume
        self.area = area

In [6]:
histories = HistoriesStorage()

vesicle = Vesicle(display_name='vesicle')
histories.register_object(vesicle)

cl_species = IonSpecies(display_name='cl',
                        internal_conc=0.01,
                        external_conc=0.02)
histories.register_object(cl_species)

na_species = IonSpecies(display_name='na',
                        internal_conc=0.03,
                        external_conc=0.04)
histories.register_object(na_species)

In [7]:
for iter_idx in range(100):
    vesicle.volume += 0.01
    vesicle.area += 0.01
    cl_species.internal_conc += 0.01
    cl_species.external_conc += 0.01
    na_species.internal_conc += 0.01
    na_species.external_conc += 0.01
    
    histories.update_histories()

In [8]:
histories.histories

{'vesicle_area': [0.11,
  0.12,
  0.13,
  0.14,
  0.15000000000000002,
  0.16000000000000003,
  0.17000000000000004,
  0.18000000000000005,
  0.19000000000000006,
  0.20000000000000007,
  0.21000000000000008,
  0.22000000000000008,
  0.2300000000000001,
  0.2400000000000001,
  0.2500000000000001,
  0.2600000000000001,
  0.27000000000000013,
  0.28000000000000014,
  0.29000000000000015,
  0.30000000000000016,
  0.31000000000000016,
  0.3200000000000002,
  0.3300000000000002,
  0.3400000000000002,
  0.3500000000000002,
  0.3600000000000002,
  0.3700000000000002,
  0.3800000000000002,
  0.39000000000000024,
  0.40000000000000024,
  0.41000000000000025,
  0.42000000000000026,
  0.43000000000000027,
  0.4400000000000003,
  0.4500000000000003,
  0.4600000000000003,
  0.4700000000000003,
  0.4800000000000003,
  0.4900000000000003,
  0.5000000000000003,
  0.5100000000000003,
  0.5200000000000004,
  0.5300000000000004,
  0.5400000000000004,
  0.5500000000000004,
  0.5600000000000004,
  0.570000

In [ ]:
k_nhe_channel = NHEChannel(flux_multiplier=-1)
h_nhe_channel = NHEChannel(flux_multiplier=2)

In [ ]:
ASOR_channel = ASORChannel()
ASOR_channel.VOLTAGE_MULTIPLIER = 1
ASOR_channel.FLUX_MULTIPLIER = 1
ASOR_channel.NERNST_MULTIPLIER = 1
ASOR_channel.VOLTAGE_SHIFT = 0

CLC_channel = CLCChannel()
CLC_channel.VOLTAGE_MULTIPLIER = 1
CLC_channel.FLUX_MULTIPLIER = 2
CLC_channel.NERNST_MULTIPLIER = 1/3
CLC_channel.VOLTAGE_SHIFT = 0

TPC_channel = TPCChannel()
TPC_channel.VOLTAGE_MULTIPLIER = -1
TPC_channel.FLUX_MULTIPLIER = 1
TPC_channel.NERNST_MULTIPLIER = 1
TPC_channel.VOLTAGE_SHIFT = 0

NHE_channel = NHEChannel()
NHE_channel.VOLTAGE_MULTIPLIER = 0
NHE_channel.FLUX_MULTIPLIER = 1
NHE_channel.NERNST_MULTIPLIER = 1
NHE_channel.VOLTAGE_SHIFT = 0

NHE_channel2 = NHEChannel()
NHE_channel2.VOLTAGE_MULTIPLIER = 0
NHE_channel2.FLUX_MULTIPLIER = -1
NHE_channel2.NERNST_MULTIPLIER = 1
NHE_channel2.VOLTAGE_SHIFT = 0

CLC_channel2 = CLCChannel()
CLC_channel2.VOLTAGE_MULTIPLIER = 1
CLC_channel2.FLUX_MULTIPLIER = -1
CLC_channel2.NERNST_MULTIPLIER = 1/3
CLC_channel2.VOLTAGE_SHIFT = 0



VATPase_channel = VATPaseChannel()
VATPase_channel.VOLTAGE_MULTIPLIER = 1
VATPase_channel.FLUX_MULTIPLIER = -1
VATPase_channel.NERNST_MULTIPLIER = -1
VATPase_channel.VOLTAGE_SHIFT = 0.27

HLeak_channel = HLeakChannel()
HLeak_channel.VOLTAGE_MULTIPLIER = -1
HLeak_channel.FLUX_MULTIPLIER = 1
HLeak_channel.NERNST_MULTIPLIER = 1
HLeak_channel.VOLTAGE_SHIFT = 0

hydrogene = IonSpecies(name='H',
                      init_vesicle_conc=7.962143411069938*1e-5,
                      exterior_conc=12.619146889603859*1e-5,
                      elementary_charge=1)
hydrogene.add_channel(channel=NHE_channel2)
hydrogene.add_channel(channel=CLC_channel2)
hydrogene.add_channel(channel=VATPase_channel)
hydrogene.add_channel(channel=HLeak_channel)


chloride.add_channel(channel=ASOR_channel)
chloride.add_channel(channel=CLC_channel)

sodium = IonSpecies(name='Na',
                    init_vesicle_conc=150*1e-3,
                    exterior_conc=10*1e-3,
                    elementary_charge=1)
sodium.add_channel(channel=TPCChannel)
sodium.add_channel(channel=NHEChannel)

potassium = IonSpecies(name='K',
                       init_vesicle_conc=5*1e-3,
                       exterior_conc=140*1e-3,
                       elementary_charge=1)
potassium.add_channel(channel=KChannel)

In [ ]:
class Simulation:
    def __init__(self,
                ...):
        self.histories = HistoresStorage()
        
    def add_hydrogen_species(hydrogen_obj):
        self.hydrogen_species = hydrogen_obj
        # No need to register, we will do it later
        
    def add_species(species_obj):
        # Something something
        self.all_species.append(species_obj)
        self.histories.register_object(species_obj)
        
    def add_channel(species_obj, channel_obj):
        species_obj.add_channel(channel_obj)
        self.histories.register_object(channel_obj)

In [ ]:
species_params = {
    'cl': {'external_conc': 1.00, 'internal_conc': 2.0, 'elementary_charge': 1},
    'na': {'external_conc': 1.00, 'internal_conc': 2.0, 'elementary_charge': -1},       
}

In [ ]:
cl_channels_params = {
    'ASOR': {
        'type': ASORChannel,
        'init_values': {
            'flux_multiplier': 1,
        },
    },
    'CLC': {
        'type': CLCChannel,
        'init_values': {
            'flux_multiplier': 2,
        },
    },      
}

na_channels_params = {
    'ASOR': {
        'type': ASORChannel,
        'init_values': {
            'flux_multiplier': 3,
        },
    },
    'CLC': {
        'type': CLCChannel,
        'init_values': {
            'flux_multiplier': 4,
        },
    }, 
}

all_channels_params = {
    'cl': cl_channels_params,
    'na': na_channels_params,
}

In [93]:
for ion_name, ion_params in species_params.items():
    ion_species = IonSpecies(display_name=ion_name, **ion_params)
    if ion_name = 'h':
        simulation.add_hydrogen_species(ion_species)
        
    simulation.add_species(ion_species)
    
    for channel_name, channel_params in all_channels_params[ion_name]:
        channel = channel_params['type'](display_name=f'{ion_name}_{channel_name}', **channel_params['init_values'])
        simulation.add_channel(ion_species, channel)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (2732168465.py, line 3)